<a href="https://colab.research.google.com/github/sarikaksstrcs/Advanced-Data-structures/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import numpy as np
import itertools
import collections


In [4]:
df = pd.read_csv('sa_data.csv')
df.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [5]:
len(df)

13871

In [6]:
from collections import defaultdict
import matplotlib.pyplot as plt
import nltk 

In [8]:
nltk.download('treebank')
nltk.download('stopwords')
nltk.download('punkt') #For Tokenisation
# nltk.download('wordnet') #dictionary
# nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [9]:
from nltk.corpus import stopwords
#from nltk.corpus import wordnet as wn
from nltk import word_tokenize,pos_tag

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score,f1_score,precision_score

import tensorflow as tf


In [11]:
from gensim.models import word2vec

In [12]:
from sklearn.metrics import recall_score,confusion_matrix

In [14]:
def preprocess_data(tweet):
  user_name = '@[A-Za-z0-9]+'
  hash_tag = '#[A-Za-z0-9]+'
  url = 'https?:\/\/[A-Za-z0-9\.\/\-]+'

  x=tweet.lower()
  x = re.sub(user_name,'',x)
  x = re.sub(hash_tag,'',x)
  x = re.sub(url,'',x)
  x = re.sub("[^a-zA-Z0-9]"," ",x)
  x = re.sub("^rt\s+",'',x)

  tokens = word_tokenize(x)
  negation_word = ['not', 'no', 'never', 'neither', 'nor', 'none', 'nobody', 'nowhere', 'nothing', 'hardly', 'scarcely', 'barely', 'cannot', 'can\'t', 'didn\'t', 'doesn\'t', 'isn\'t', 'wasn\'t', 'won\'t', 'wouldn\'t', 'shouldn\'t', 'couldn\'t', 'haven\'t', 'hasn\'t', 'hadn\'t']


  stop_word = set(stopwords.words("english"))

  stops  = [x for x in stop_word if not x in negation_word]
  words = [x for x in tokens if not x in stops]

  return(" ".join(words))



In [17]:
df['text'] = df['text'].apply(lambda x: preprocess_data(x))
df.head()

,text,sentiment
0,everyone feel climate change question last nig...,Neutral
1,catch full last night scott best lines 90 seconds,Positive
2,mention tamir rice held cleveland wow,Neutral
3,carly fiorina trending hours debate men comple...,Positive
4,w delivered highest ratings history presidenti...,Positive


In [29]:
review = df['text'].tolist()
sent = df['sentiment'].tolist()

le = LabelEncoder()

y = le.fit_transform(sent)
labels = tf.keras.utils.to_categorical(y)

y

array([1, 2, 1, ..., 2, 0, 2])

In [30]:
labels

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [31]:
x_train,x_test,y_train,y_test = train_test_split(review,y,test_size = 0.2,random_state = 42)

In [32]:
vocab_size = 4000 #incorrect
embedding_diam  = 100

max_len = 0

for i in range(len(df['text'])):
  x = word_tokenize(df['text'][i])
  if len(x)>max_len :
    max_len = len(x)
max_len

28

In [33]:
oov_tok = "<OOV"

In [35]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words= vocab_size,oov_token = oov_tok)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index
word_index

{'<OOV': 1,
 'trump': 2,
 'fox': 3,
 'amp': 4,
 'debate': 5,
 'news': 6,
 'night': 7,
 'last': 8,
 'candidates': 9,
 'like': 10,
 'people': 11,
 'bush': 12,
 'jeb': 13,
 'one': 14,
 'gop': 15,
 'would': 16,
 'get': 17,
 'think': 18,
 'president': 19,
 'god': 20,
 'donald': 21,
 'chris': 22,
 'cruz': 23,
 'republican': 24,
 'ask': 25,
 'need': 26,
 'want': 27,
 'rubio': 28,
 'wallace': 29,
 'women': 30,
 'really': 31,
 'question': 32,
 'said': 33,
 'carson': 34,
 'questions': 35,
 'tonight': 36,
 'watching': 37,
 'know': 38,
 'next': 39,
 'time': 40,
 'candidate': 41,
 'right': 42,
 'huckabee': 43,
 'see': 44,
 'job': 45,
 'anyone': 46,
 'fair': 47,
 'megyn': 48,
 'trying': 49,
 'face': 50,
 'stage': 51,
 'say': 52,
 'america': 53,
 'good': 54,
 'hear': 55,
 'got': 56,
 'balanced': 57,
 'take': 58,
 'american': 59,
 'walker': 60,
 'never': 61,
 'paul': 62,
 'us': 63,
 'party': 64,
 'watch': 65,
 'great': 66,
 'go': 67,
 '2': 68,
 'talk': 69,
 'set': 70,
 'via': 71,
 'going': 72,
 'men':

In [38]:
training_sequences =tokenizer.texts_to_sequences(x_train)
training_padded = tf.keras.preprocessing.sequence.pad_sequences(training_sequences,maxlen = max_len,padding = 'pre') 

In [39]:
test_sequences =tokenizer.texts_to_sequences(x_test)
test_padded = tf.keras.preprocessing.sequence.pad_sequences(training_sequences,maxlen = max_len)